In [4]:
# ML Zoomcamp Module 3 Homework - Tristan Kilper

In [19]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [3]:
# Dataset

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv")

In [5]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [6]:
# Data Preparation

In [7]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [8]:
df[['lead_source','industry','employment_status','location']] = df[['lead_source','industry','employment_status','location']].fillna('NA')

In [9]:
df[['annual_income']] = df[['annual_income']].fillna(0.0)

In [10]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [11]:
# Q1

In [12]:
df[['industry']].value_counts()

industry     
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

In [13]:
# Q2

In [16]:
correlation_matrix = df[['lead_score','interaction_count','number_of_courses_viewed','annual_income']].corr()
correlation_matrix

,lead_score,interaction_count,number_of_courses_viewed,annual_income
lead_score,1.000000,0.009888,-0.004879,0.015610
interaction_count,0.009888,1.000000,-0.023565,0.027036
number_of_courses_viewed,-0.004879,-0.023565,1.000000,0.009770
annual_income,0.015610,0.027036,0.009770,1.000000


In [17]:
# Split the data